## Notebook with ipywidget examples

### DVC - 24/11/2020

This notebook contains basic ipywidgets for user interaction.
It also has a small game
This notebook should be robust against platforms and evolution

For interactions involving plotting cfr. ipywidgets-mpl

In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
import IPython.display as ipd
import numpy as np

In [2]:
# widget1: integer slider
slider1 = widgets.IntSlider(max=20)
ipd.display(slider1)
def my_func(x):
    print(2*x)

IntSlider(value=0, max=20)

In [3]:
# widget2: animation by Play
play2 = widgets.Play(
    interval=1000, # interval in msec between animation frame
    value=50,
    min=0,
    max=100,
    step=1,
    description="Press play",
    disabled=False
)
slider2 = widgets.IntSlider()
widgets.jslink((play2, 'value'), (slider2, 'value'))
widgets.HBox([play2, slider2])

In [4]:
# widget 3: printing text
@interact( text='', times=(0,5) )
def printIt(text,times):
    print( text * times )

interactive(children=(Text(value='', description='text'), IntSlider(value=2, description='times', max=5), Outp…

In [5]:
# This works as expected in Jupyter Notebook, but not in Jupyter lab !
button1 = widgets.Button(description="B1")

def printMe(b):
        print("Printing something")
button1.on_click(printMe)
button1

Button(description='B1', style=ButtonStyle())

In [6]:
# some globals as needed
global user_history
user_history = [1,0]
target_score = 5

#buttons
btn0 = widgets.Button( description= '0' )
btn1 = widgets.Button( description= '1' )
btnS = widgets.Button( description= 'START' )

def click_zero(b):
    update_game(0)
btn0.on_click( click_zero )
def click_one(b):
    update_game(1)
btn1.on_click( click_one )
def click_start(b):
    update_game(0,start=True)
btnS.on_click( click_start )
btn0.disabled = True
btn1.disabled = True
btn0.button_style ='info'
btn1.button_style ='info'
btnS.button_style ='warning'
# score board
user_score = widgets.IntProgress(
    value=0,
    min=0, max=target_score,step=1,
    description='You',
    bar_style='success',
    orientation='horizontal'
    )
bot_score = widgets.IntProgress(
    value=0,
    min=0, max=target_score,step=1,
    description='Bot',
    bar_style='danger',
    orientation='horizontal'
    )
scoreboard= widgets.VBox((user_score,bot_score))
final_msg = widgets.HTML("<h1 style='color:green'> You win </h1>")
final_msg.layout.visibility = "hidden"
# layout
game = widgets.VBox( ( widgets.HBox((scoreboard,final_msg)),
                   widgets.HBox((btn0,btn1,btnS))
             ))

In [7]:
#
# game rules
#
# note only user_history had to be defined as a global
# all widgets behave by nature as such

def update_game(user_choice,start=False):
    global user_history

    if( start ):
        user_history = [1,0]
        final_msg.value = "<h1 style='color:green'> You win </h1>"
        final_msg.layout.visibility = "hidden"
        user_score.value = 0
        bot_score.value = 0
        btn0.disabled = False
        btn1.disabled = False
        btnS.disabled = True
        btnS.button_style = ''
        return
    prob = sum(user_history)/len(user_history)
    comp_choice = np.random.binomial(1,prob,1)[0]
    user_history.append( user_choice )
    # print("sequence length: ",len(user_history)-2)
    if comp_choice == user_choice:
        bot_score.value += 1
    else:
        user_score.value += 1
    if user_score.value == target_score or bot_score.value == target_score:
        if bot_score.value == target_score:
            final_msg.value = "<h1 style='color:red'>Game Over</h1>"
        final_msg.layout.visibility = "visible"
        btn0.disabled = True
        btn1.disabled = True
        btnS.disabled = False
        btnS.button_style = 'warning'
    return

In [8]:
game

In [11]:
print(bot_score.value)
display(final_msg)

5


HTML(value="<h1 style='color:red'>Game Over</h1>", layout=Layout(visibility='visible'))

In [12]:
# widget 4: figure update
fig = plt.figure()
@interact( m=(-2.,2.), b=(-3,3,0.5))
def f(m,b):
    #plt.figure(2)
    x = np.linspace(-10,10,num=1000)
    plt.plot(x, m*x+b )
    plt.ylim(-5,5)
    plt.show()

NameError: name 'plt' is not defined

In [ ]:
# widget3 @interact decorator
# control amplitude, phase .. of a simple function generator
#
def generate(type='sin', freq=200.0, Fs=8000.0, Tmax=0.5):
    t = np.linspace(0.0, Tmax, int(Tmax*Fs), endpoint=False)
    if type == 'sin':
        x = np.sin(2.0*np.pi*freq*t)
    elif type == 'square':
        x = signal.square(2.0*np.pi*freq*t)
    elif type == 'sawtooth':
        x = signal.sawtooth(2.0*np.pi*freq*t)
    else:
        print( 'signal: Unrecognized signal type')
    return x, t

signal_types = [ 'sin', 'square', 'sawtooth' ]
# alternative to Dropdown is to use RadioButtons
wg_signal = widgets.Dropdown(options=signal_types,
                             value='sin',description="Signal")
wg_amp = widgets.FloatSlider(value=1.0,step=0.05,max=1.0,description='Amplitude')
wg_freq = widgets.FloatSlider(value=200.0,step=5.0,min=50.0,max=500.0,description='Frequency')
wg_Tmax = widgets.FloatSlider(value=0.6,step=0.1,min=0.1,max=2.0,description='Duration')
wg_check = widgets.Checkbox(description='Display Play Button',value=True)
Fs=8000
plt.close('all')
fig = plt.figure()

# in this example we use interact and full replotting
# this seems to work well with ALL backends
@interact(t=wg_signal,f=wg_freq,A=wg_amp,T=wg_Tmax, v=wg_check)
def plot_fn(t,f,A,T,v):
    x1, t1 = generate(freq=f,type=t,Tmax=T)
    y1=A*x1
    plt.cla()
    plt.plot(t1,y1)
    plt.ylim(-1,1)
    plt.show()
    if(v):
        ipd.display(ipd.Audio(data=y1,rate=Fs,normalize=False))
    return

In [ ]:
plt.close('all')

In [ ]:
b1 = widgets.Button(description='button 1')
b2 = widgets.Button(description='button 2')
b3 = widgets.Button(description='button 3')
def make_boxes():
    vbox1 = widgets.VBox([widgets.Label('Left'), b1, b2])
    vbox2 = widgets.VBox([widgets.Label('Right'), b3 ])
    return vbox1, vbox2
 
vbox1, vbox2 = make_boxes()


def b1_clicked(b):
    print("Button 1 clicked.")
b1.on_click(b1_clicked)

 
widgets.HBox([vbox1, vbox2])

In [ ]:
x=range(0,10)
xslider = widgets.FloatSlider(value=0.,step=1.,max=10.0,description='Range')
fig = plt.figure(1,figsize=(15,4),clear=True)
plt.plot(x)
plt.show()
print(fig.number)
@interact(x=xslider)
def plot_func(x):
    f2=plt.figure(1)
    #print(f2)
    plt.vlines(x,0,10,linestyles='solid',colors='red')
    plt.xlim(0,10)
    plt.draw()


In [ ]:
# tracking change with the observe mechanism
# only works in widget backend
#%matplotlib inline
fig,ax = plt.subplots()
line, = ax.plot(5*np.sin(range(1,100)))
def show_change(change):
    #print(change.new)
    # [l.remove() for l in ax.lines]
    line.set_ydata(change.new*np.sin(range(1,100)))
    fig.canvas.draw()
    #plt.show()

int_slider = widgets.IntSlider(value=4, min=0, max=8)
int_slider.observe(show_change, 'value')
int_slider.value = 5
int_slider

In [ ]:
# set up plot
fig, ax = plt.subplots(figsize=(6, 4))
ax.set_ylim([-4, 4])
ax.grid(True)
 
# generate x values
x = np.linspace(0, 2 * np.pi, 100)
 
def my_sine(x, w, amp, phi):
    """
    Return a sine for x with angular frequeny w and amplitude amp.
    """
    return amp*np.sin(w * (x-phi))


@interact(w=(0, 10, 1), amp=(0, 4, .1), phi=(0, 2*np.pi+0.01, 0.01))
def update(w = 1.0, amp=1, phi=0):
    """Remove old lines from plot and plot new one"""
    [l.remove() for l in ax.lines]
    ax.plot(x, my_sine(x, w, amp, phi), color='C0')
    #plt.show()

In [ ]:
rec_play = widgets.Dropdown(
    options={'REC','PLAY'},
    description='ACTION',
)

In [ ]:
progress_bar = widgets.IntProgress(
    value=5,
    min=0, max=10,step=1,
    description='Loading',
    bar_style='',
    orientation='horizontal'
    )
progress_bar

In [ ]:
# this is probably somewhat cleaner code from the previous
button1 = widgets.Button(description="B1")
output = widgets.Output()
ipd.display(button1,output)
def printMe(b):
    with output:
        print("Printing something")
button1.on_click(printMe)
#button1

In [ ]:
@interact_manual( m=(-2.,2.), b=(-3,3,0.5))
def f(m,b):
    plt.figure(2)
    x = np.linspace(-10,10,num=1000)
    plt.plot(x, m*x+b )
    plt.ylim(-5,5)
    plt.show()

In [ ]:
%matplotlib widget
# set up plot
fig, ax = plt.subplots(figsize=(6, 4))
ax.set_ylim([-4, 4])
ax.grid(True)

# generate x values
x = np.linspace(0, 2 * np.pi, 100)


def my_sine(x, w, amp, phi):
    """
    Return a sine for x with angular frequeny w and amplitude amp.
    """
    return amp*np.sin(w * (x-phi))


@widgets.interact(w=(0, 10, 1), amp=(0, 4, .1), phi=(0, 2*np.pi+0.01, 0.01))
def update(w = 1.0, amp=1, phi=0):
    """Remove old lines from plot and plot new one"""
    """As you plot again each time the color of the line goes through its normal sequence """
    [l.remove() for l in ax.lines]
    ax.plot(x, my_sine(x, w, amp, phi))

In [ ]:
%matplotlib widget
# set up plot
fig, ax = plt.subplots(figsize=(6, 4))
ax.set_ylim([-4, 4])
ax.grid(True)

# generate x values
x = np.linspace(0, 2 * np.pi, 100)
line, = ax.plot(x, my_sine(x, 0, 0, 0))

def my_sine(x, w, amp, phi):
    """
    Return a sine for x with angular frequeny w and amplitude amp.
    """
    return amp*np.sin(w * (x-phi))


@widgets.interact(w=(0, 10, 1), amp=(0, 4, .1), phi=(0, 2*np.pi+0.01, 0.01))
def update(w = 1.0, amp=1, phi=0):
    """Remove old lines from plot and plot new one"""
    """As you plot again each time the color of the line goes through its normal sequence """
    newline = my_sine(x, w, amp, phi)
    ax.set_ydata(newline)